In [147]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import numpy as np
from datetime import timedelta
from haversine import haversine
from random import randrange
from datetime import timedelta
from datetime import datetime

from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

import datetime as dt

from datetime import date
import holidays

In [365]:
data = pd.read_csv('data_2022_processed.csv')
random_data = pd.read_csv('random_data_2022_processed.csv')

In [88]:
may_data = pd.read_csv('may_data.csv')

# Обучение модели

## Подготовка данных

In [89]:
random_data.accept_datetime = pd.to_datetime(random_data.accept_datetime)
may_data.accept_datetime = pd.to_datetime(may_data.accept_datetime)

In [90]:
may_data['hour'] = may_data.accept_datetime.apply(lambda x: x.hour)
random_data['hour'] = random_data.accept_datetime.apply(lambda x: x.hour)

In [370]:
data.accept_datetime = pd.to_datetime(data.accept_datetime)
data['hour'] = data.accept_datetime.apply(lambda x: x.hour)

In [91]:
random_data = random_data.rename(columns={'latitude':'latitude_x', 'longitude':'longitude_x'})

In [92]:
may_random_data = pd.DataFrame()

for i, val in may_data.groupby('substation').accept_datetime.count().reset_index().iterrows():
    
    temp_may_random_data = random_data[(random_data.substation == val['substation'])
                                       &(random_data.accept_datetime > may_data.accept_datetime.min())].sample(val['accept_datetime'])
    temp_may_random_data[['latitude_y', 'longitude_y', 'dist']] \
    = may_data.loc[may_data.substation == val['substation'], ['latitude_y', 'longitude_y', 'dist']].values
    
    may_random_data = pd.concat([may_random_data,
                                 temp_may_random_data],
                                ignore_index=True)

In [93]:
for i in range(len(may_random_data)):
    may_random_data.latitude_y.iloc[i] += np.random.rand() / 1000
    may_random_data.longitude_y.iloc[i] += np.random.rand() / 1000
    may_random_data.dist.iloc[i] += np.random.rand()*10

/Users/hare/opt/anaconda3/envs/champ/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [121]:
weather = pd.read_csv('weather_2022.csv', encoding='utf-8', skiprows=6, sep=';', index_col=0)

weather_columns = weather.columns[1:]
weather = weather.drop(weather.columns[-1], axis=1)
weather.columns = weather_columns

weather = weather.reset_index().rename(columns={'index':'datetime'})
weather['datetime'] = pd.to_datetime(weather['datetime'])
weather = weather.sort_values('datetime').reset_index(drop=True)

weather = weather[['datetime', 'T', 'Tg', 'Po', 'Pa', 'U', 'ff3']]

weather.datetime = pd.to_datetime(weather.datetime)

In [123]:
weather_for_data = pd.DataFrame()

for adt in tqdm(may_random_data['accept_datetime']):
    weather_for_data = pd.concat([weather_for_data, weather[weather.datetime <= adt].iloc[-1:]],
                                 ignore_index=True)
    
may_random_data[weather_for_data.columns[1:]] = weather_for_data[weather_for_data.columns[1:]]

100%|██████████████████████████████████████| 1372/1372 [00:01<00:00, 980.22it/s]


In [125]:
weather_for_data = pd.DataFrame()

for adt in tqdm(may_data['accept_datetime']):
    weather_for_data = pd.concat([weather_for_data, weather[weather.datetime <= adt].iloc[-1:]],
                                 ignore_index=True)
    
may_data[weather_for_data.columns[1:]] = weather_for_data[weather_for_data.columns[1:]]

100%|██████████████████████████████████████| 1372/1372 [00:01<00:00, 990.69it/s]


In [382]:
data.accept_datetime = data.accept_datetime.apply(lambda x: pd.to_datetime(str(x)[:-6]))

In [383]:
weather_for_data = pd.DataFrame()

for adt in tqdm(data['accept_datetime']):
    weather_for_data = pd.concat([weather_for_data, weather[weather.datetime <= adt].iloc[-1:]],
                                 ignore_index=True)
    
data[weather_for_data.columns[1:]] = weather_for_data[weather_for_data.columns[1:]]

100%|████████████████████████████████████| 37258/37258 [00:39<00:00, 950.76it/s]


## Предсказание вызова

In [356]:
may_data['call_target'] = 1
may_random_data['call_target'] = 0

train_data = pd.concat([may_data, may_random_data])[[
    'hour', 'weekday', 'day', 'week', 'month', 'holiday', 
    'T', 'Tg', 'Po', 'Pa', 'U', 'ff3', 'substation', 'call_target',
    'latitude_x', 'longitude_x', 'latitude_y', 'longitude_y', 'dist'
]]



cat_features = ['substation', 'holiday']
train_data[cat_features] = train_data[cat_features].fillna('')

In [357]:
X_train, X_val, y_train, y_val = train_test_split(train_data.drop(['call_target'], axis=1),
                                                  train_data['call_target'],
                                                  stratify=train_data.call_target,
                                                  # stratify=train_data['call_target'].astype(str) + '+' + train_data['substation'],
                                                  test_size=0.25)

In [358]:
model = CatBoostClassifier(verbose=100, cat_features=cat_features,
                           loss_function='MultiClassOneVsAll'
                          )
model.fit(X_train, y_train, eval_set=(X_val, y_val))

0:	learn: 0.6795429	test: 0.6801924	best: 0.6801924 (0)	total: 5.49ms	remaining: 5.49s
100:	learn: 0.3502050	test: 0.3799159	best: 0.3799159 (100)	total: 336ms	remaining: 2.99s
200:	learn: 0.3077293	test: 0.3563374	best: 0.3563374 (200)	total: 683ms	remaining: 2.71s
300:	learn: 0.2733137	test: 0.3368174	best: 0.3368174 (300)	total: 1.08s	remaining: 2.52s
400:	learn: 0.2451141	test: 0.3222202	best: 0.3221897 (399)	total: 1.45s	remaining: 2.17s
500:	learn: 0.2211669	test: 0.3063838	best: 0.3063309 (499)	total: 1.83s	remaining: 1.82s
600:	learn: 0.2020420	test: 0.2939445	best: 0.2939445 (600)	total: 2.15s	remaining: 1.43s
700:	learn: 0.1890799	test: 0.2888283	best: 0.2888283 (700)	total: 2.58s	remaining: 1.1s
800:	learn: 0.1758952	test: 0.2825294	best: 0.2824610 (796)	total: 2.97s	remaining: 738ms
900:	learn: 0.1655567	test: 0.2799365	best: 0.2799250 (899)	total: 3.34s	remaining: 367ms
999:	learn: 0.1562933	test: 0.2768069	best: 0.2767194 (996)	total: 3.7s	remaining: 0us

bestTest = 0.276

In [359]:
model.save_model('models/call')

In [360]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

           0       0.90      0.87      0.88       343
           1       0.87      0.90      0.89       343

    accuracy                           0.88       686
   macro avg       0.89      0.88      0.88       686
weighted avg       0.89      0.88      0.88       686



In [361]:
fip = pd.DataFrame()
fip['feature'] = model.feature_names_
fip['importance'] = model.get_feature_importance()
fip = fip.sort_values('importance', ascending=False)

In [362]:
fip

,feature,importance
17,dist,41.964064
15,latitude_y,13.327498
16,longitude_y,10.267830
13,latitude_x,6.196968
12,substation,5.744491
0,hour,5.680550
14,longitude_x,4.880584
2,day,1.989463
6,T,1.775277
8,Po,1.687345


## Инференс предсказаний вызова

In [349]:
# left_up = (56.395527, 43.787622)
# left_low = (56.194032, 43.787622)

# right_up = (56.395527, 44.087140)
# right_low = (56.194032, 44.087140)

squares = []

for i in range(0, 50):
    for j in range(0, 50):
        squares.append((56.194032 + i * (56.395527 - 56.194032) / 50,
                        43.787622 + j * (44.227140 - 43.787622) / 50))

In [351]:
test = pd.DataFrame()

test['coords'] = squares
test['latitude_y'] = test.coords.apply(lambda x: x[0])
test['longitude_y'] = test.coords.apply(lambda x: x[1])

test['datetime'] = dt.datetime.now()
test['hour'] = test.datetime.apply(lambda x: x.hour)
test['weekday'] = test.datetime.apply(lambda x: x.weekday())
test['day'] = test.datetime.apply(lambda x: x.day)
test['week'] = test.datetime.apply(lambda x: x.week)
test['month'] = test.datetime.apply(lambda x: x.month)
test['holiday'] = test.datetime.apply(lambda x: holidays.RUS().get(x))
test['holiday'] = test['holiday'].fillna('')

for col in weather.columns[1:]:
    test[col] = weather.loc[weather.datetime <= test['datetime'].iloc[0], col].iloc[-1]

In [352]:
substations = pd.read_excel('substations.xlsx').rename(columns={'name':'substation'})
substations['latitude_x'] = substations.coords.apply(lambda x: float(x.split(',')[0]))
substations['longitude_x'] = substations.coords.apply(lambda x: float(x.split(',')[1]))
substations.coords = substations[['latitude_x', 'longitude_x']].apply(lambda x: tuple([x[0], x[1]]), axis=1)

In [353]:
temp = pd.DataFrame()
for i in range(len(substations)):
    temp[substations.substation.iloc[i]] = test.coords.apply(lambda x: haversine(x, substations.coords.iloc[i]))
    
temp = temp.T

nearest_subs = []
for i in range(temp.shape[1]):
    nearest_subs.append(temp[i].sort_values().index[0])
    
test['substation'] = nearest_subs
test = pd.merge(test, substations, on='substation', how='left')
test['dist'] = test[['coords_x', 'coords_y']].apply(lambda x: haversine(x[0], x[1]), axis=1)
test = test.drop('coords_y', axis=1).rename(columns={'coords_x': 'coords'})

In [355]:
model.predict_proba(test[model.feature_names_])[:, -1]

In [269]:
test_res = pd.DataFrame()
test_res['substation'] = test.substation
test_res['latitude'] = test.latitude_y
test_res['longitude'] = test.longitude_y
test_res['call_target'] = model.predict_proba(test[model.feature_names_])[:, -1]

In [270]:
test_res.to_csv('test_res.csv', index=False)

In [271]:
test_res

,substation,latitude,longitude,call_target
0,ПСМП №1,56.194032,43.787622,0.057267
1,ПСМП №1,56.194032,43.796412,0.092618
2,ПСМП №1,56.194032,43.805203,0.077473
3,ПСМП №1,56.194032,43.813993,0.078966
4,ПСМП №1,56.194032,43.822783,0.101174
...,...,...,...,...
2495,Борская ЦРБ СМП Затон ППК,56.391497,44.183188,0.015975
2496,Борская ЦРБ СМП Затон ППК,56.391497,44.191979,0.013621
2497,Борская ЦРБ СМП Затон ППК,56.391497,44.200769,0.011822
2498,Борская ЦРБ СМП Затон ППК,56.391497,44.209559,0.011084


In [41]:
# substation
# данные по времени
# погода (через яндекс апи, брать только НН, каждую минуту)
# road_time - накинуть эвристику

In [272]:
# import datetime as dt

# # Save the current time to a variable ('t')
# t = dt.datetime.now()

# while True:
#     delta = dt.datetime.now()-t
#     if delta.seconds >= 60:
#         print("1 Min")
#         # Update 't' variable to new time
#         t = dt.datetime.now()

## Предсказание тяжести

In [445]:
train_data = data[[
    'hour', 'weekday', 'day', 'week', 'month', 'holiday', 
    'T', 'Tg', 'Po', 'Pa', 'U', 'ff3', 'substation',
    'who', 'calling', 'type', 'occasion', 'result'
]].copy()

cat_features = ['substation', 'holiday', 'who', 'calling', 'type', 'occasion']
train_data[cat_features] = train_data[cat_features].fillna('')

In [446]:
first_occasions = pd.read_csv('Первичные признаки - Лист1.csv')
first_occasions = first_occasions.rename(columns={'Тяжесть (0 - норма, 1 - тяжелое)':'heaviness',
                                                  'Признаки':'occasion'})
first_occasions = first_occasions[['occasion', 'heaviness']]
first_occasions = first_occasions[first_occasions.heaviness.notnull()]

In [447]:
train_data = pd.merge(train_data, first_occasions, how='left', on='occasion')

In [448]:
train_data.loc[train_data['type'] == 'неотложное состояние', 'heaviness'] = 1

In [449]:
data_for_demo = data[train_data.heaviness.notnull()].reset_index(drop=True)

In [450]:
train_data = train_data[train_data.heaviness.notnull()].reset_index(drop=True)

In [389]:
# targets = ['result', 'heaviness']

In [390]:
X_train, X_val, y_train, y_val = train_test_split(train_data[train_data.heaviness.notnull()].drop(['result', 'heaviness'], axis=1),
                                                  train_data[train_data.heaviness.notnull()]['heaviness'],
                                                  stratify=train_data[train_data.heaviness.notnull()]['heaviness'],
                                                  test_size=0.15)

In [391]:
model = CatBoostClassifier(verbose=100, cat_features=cat_features)
model.fit(X_train, y_train, eval_set=(X_val, y_val))

Learning rate set to 0.073386
0:	learn: 0.5253449	test: 0.5253971	best: 0.5253971 (0)	total: 20.7ms	remaining: 20.7s
100:	learn: 0.0015805	test: 0.0003141	best: 0.0003141 (100)	total: 1.75s	remaining: 15.5s
200:	learn: 0.0013964	test: 0.0002579	best: 0.0002579 (200)	total: 3.26s	remaining: 12.9s
300:	learn: 0.0013835	test: 0.0002594	best: 0.0002579 (200)	total: 4.44s	remaining: 10.3s
400:	learn: 0.0013832	test: 0.0002592	best: 0.0002579 (200)	total: 5.7s	remaining: 8.51s
500:	learn: 0.0013831	test: 0.0002592	best: 0.0002579 (200)	total: 6.84s	remaining: 6.82s
600:	learn: 0.0013827	test: 0.0002591	best: 0.0002579 (200)	total: 8.02s	remaining: 5.33s
700:	learn: 0.0013822	test: 0.0002589	best: 0.0002579 (200)	total: 9.19s	remaining: 3.92s
800:	learn: 0.0013818	test: 0.0002589	best: 0.0002579 (200)	total: 10.4s	remaining: 2.58s
900:	learn: 0.0013812	test: 0.0002588	best: 0.0002579 (200)	total: 11.5s	remaining: 1.27s
999:	learn: 0.0013807	test: 0.0002587	best: 0.0002579 (200)	total: 12.7s	r

In [392]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       361
         1.0       1.00      1.00      1.00      4927

    accuracy                           1.00      5288
   macro avg       1.00      1.00      1.00      5288
weighted avg       1.00      1.00      1.00      5288



In [393]:
model.save_model('models/heaviness')

In [403]:
good_results = list(train_data.result.value_counts().iloc[:13].index)

In [404]:
X_train, X_val, y_train, y_val = train_test_split(train_data[train_data.result.isin(good_results)].drop(['result', 'heaviness'], axis=1),
                                                  train_data[train_data.result.isin(good_results)]['result'],
                                                  stratify=train_data[train_data.result.isin(good_results)]['result'],
                                                  test_size=0.15)

In [405]:
model = CatBoostClassifier(verbose=100, cat_features=cat_features)
model.fit(X_train, y_train, eval_set=(X_val, y_val))

Learning rate set to 0.115573
0:	learn: 1.9629577	test: 1.9635362	best: 1.9635362 (0)	total: 300ms	remaining: 4m 59s
100:	learn: 0.5602790	test: 0.5741797	best: 0.5741797 (100)	total: 1m 35s	remaining: 14m 9s
200:	learn: 0.5388497	test: 0.5718275	best: 0.5716454 (182)	total: 3m 25s	remaining: 13m 36s
300:	learn: 0.5207200	test: 0.5720133	best: 0.5712617 (242)	total: 5m 13s	remaining: 12m 8s
400:	learn: 0.5038742	test: 0.5731282	best: 0.5712617 (242)	total: 6m 55s	remaining: 10m 20s
500:	learn: 0.4905263	test: 0.5744305	best: 0.5712617 (242)	total: 8m 44s	remaining: 8m 42s
600:	learn: 0.4762553	test: 0.5756898	best: 0.5712617 (242)	total: 10m 30s	remaining: 6m 58s
700:	learn: 0.4643986	test: 0.5766239	best: 0.5712617 (242)	total: 12m 5s	remaining: 5m 9s
800:	learn: 0.4528435	test: 0.5782388	best: 0.5712617 (242)	total: 13m 48s	remaining: 3m 25s
900:	learn: 0.4425088	test: 0.5802355	best: 0.5712617 (242)	total: 15m 22s	remaining: 1m 41s
999:	learn: 0.4324830	test: 0.5816970	best: 0.57126

In [406]:
print(classification_report(y_val, model.predict(X_val)))

                                                         precision    recall  f1-score   support

                             больной не найден на месте       0.00      0.00      0.00         2
                                          вызов отменен       0.00      0.00      0.00         1
                                   доставлен в больницу       1.00      0.03      0.06        31
              оказана помощь, больной оставлен на месте       0.76      0.78      0.77      3010
                                        отказ от помощи       0.00      0.00      0.00         4
отказ от транспортировки для госпитализации в стационар       0.67      0.66      0.66      2114
                                    передан бригаде СПЦ       0.00      0.00      0.00         1
                       смерть в присутствии бригады СМП       0.00      0.00      0.00         2
                          смерть до приезда бригады СМП       0.76      0.80      0.78       120

                            

/Users/hare/opt/anaconda3/envs/champ/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hare/opt/anaconda3/envs/champ/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hare/opt/anaconda3/envs/champ/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [407]:
model.save_model('models/result')

In [ ]:
# внимание 56.294379, 44.052212
# вызов 56.292367, 44.032201
# первая 56.290812, 44.036148 - останется на более тяжелый случай
# вторая 56.302092, 44.011915 - уедет на вызов

In [461]:
data_for_demo = pd.merge(may_data, first_occasions, how='left', on='occasion')
data_for_demo.loc[data_for_demo['type'] == 'неотложное состояние', 'heaviness'] = 1
data_for_demo = data_for_demo[data_for_demo.heaviness.notnull()].reset_index(drop=True)

In [463]:
data_for_demo.heaviness.value_counts()

1.0    1265
0.0      63
Name: heaviness, dtype: int64

In [471]:
data_for_demo[['adress', 'latitude_y', 'longitude_y']]

,adress,latitude_y,longitude_y
0,"г.Нижний Новгород, ул. Гордеевская, д.58, к***",56.331718,43.928174
1,"г.Нижний Новгород, ул. Гордеевская, д.58, кв***",56.331718,43.928174
2,"г.Нижний Новгород, ул. 40 лет Победы, д.7, кв***",56.225913,43.947479
3,"г.Нижний Новгород, ул. Львовская, д.7, к***",56.266752,43.875147
4,"г.Нижний Новгород, наб. им Федоровского, д.6, ...",56.326018,43.987697
...,...,...,...
1323,"г.Нижний Новгород, ул. Чаадаева, д.14, кв***",56.329427,43.849105
1324,"г.Нижний Новгород, ул. Циолковского, д.25а, кв***",56.340929,43.842870
1325,"г.Нижний Новгород, ул. Металлистов, д.111",56.312050,43.853920
1326,"г.Нижний Новгород, ул. Канашская, д.2А, к***",56.307142,44.039071


In [473]:
cols = list(train_data.columns)
cols.append('adress')
cols.append('latitude_y')
cols.append('longitude_y')
data_for_demo[cols].to_csv('data_for_demo.csv', index=False)

In [426]:
train_data.iloc[20211:20212].to_csv('demo_df.csv', index=False)

In [436]:
train_data.iloc[6919:6920].to_csv('demo2_df.csv', index=False)

In [441]:
pd.to_datetime('2022-04-30'+' '+str(dt.datetime.now().time()))

Timestamp('2022-04-30 06:58:41.372955')

In [431]:
model.predict(train_data[model.feature_names_].iloc[20211:20212])[0][0]

'отказ от транспортировки для госпитализации в стационар'

In [432]:
hmodel = CatBoostClassifier().load_model('models/heaviness')

In [434]:
hmodel.predict_proba(train_data[hmodel.feature_names_].iloc[20211:20212])[0, -1]

0.00026170060242840866

In [435]:
pd.read_csv('demo_df.csv')

,hour,weekday,day,week,month,holiday,T,Tg,Po,Pa,U,ff3,substation,who,calling,type,occasion,result,heaviness
0,5,4,11,10,3,NaN,12.1,NaN,738.7,-1.1,61,10.0,ПСМП №7,NaN,Первичный,внезапное заболевание,Плохо ребенку (причина неизвестна),"оказана помощь, больной оставлен на месте",0.0


In [479]:
temp = pd.read_csv('data_for_demo.csv')
cat_features = ['substation', 'holiday', 'who', 'calling', 'type', 'occasion']
temp[cat_features] = temp[cat_features].fillna('')
# temp['Прогнозируемый Результат'] = result_model.predict(temp[result_model.feature_names_])[0][0]
# temp['Прогнозируемая Тяжесть'] = round(heavy_model.predict_proba(temp[heavy_model.feature_names_])[0, -1], 2)
temp = temp.rename(columns={'occasion': 'Повод', 'calling': 'Вызов', 'type': 'Вид', 'substation': 'Подстанция',
                           'adress':'Адрес'})

array([ 183,   73,  220, 1320])